# 使用 MCP 建置代理程式：AI 與外部資源的無縫整合

> **建立者：** :robot: CURSOR

## 簡介

模型內容協定（Model Context Protocol, MCP）是一個開放協定，旨在標準化應用程式如何為大型語言模型（LLMs）提供內容。將 MCP 想像成 AI 應用程式的 USB-C 連接埠 - 就像 USB-C 提供標準化的方式來連接裝置與各種周邊設備，MCP 提供了標準化的方式來連接 AI 模型與不同的資料來源和工具。

本教學將引導您實作 MCP 於您的 AI 代理程式應用中，展示如何透過提供對外部資源、工具和資料來源的無縫存取來增強代理程式的能力。

## 為什麼 MCP 對代理程式很重要

傳統的 AI 模型與外部資源連接方法通常涉及為每個資料來源或工具進行客製化整合。這導致：

- **整合複雜性**：每個新的資料來源都需要獨特的實作
- **擴展性問題**：新增工具變得越來越困難
- **維護負擔**：對一個整合的更新可能會破壞其他整合

MCP 透過提供標準化協定來解決這些挑戰，該協定能夠：

- **統一存取**：多個資料來源和工具的單一介面
- **即插即用擴展**：輕鬆新增新功能
- **狀態化通訊**：AI 與資源之間的即時雙向通訊
- **動態發現**：AI 可以即時找到並使用新工具

以下是強調官方 MCP 伺服器範例的簡潔段落：

## 官方 MCP 伺服器範例

MCP 社群維護了一系列參考伺服器實作，展示最佳實務並演示各種整合模式。這些官方範例可在 [MCP 伺服器](https://github.com/modelcontextprotocol/servers/tree/main/src) 取得，為想要建立自己 MCP 伺服器的開發者提供寶貴的起點。

## 我們將建置的內容

在本教學中，我們將實作：

1. **建置您的 MCP 伺服器並使用它**：建置一個具有客製化工具的 MCP 伺服器並連接到 Claude Desktop
2. **客製化工具啟用代理程式**：建立一個可以透過 MCP 使用外部工具的客製化代理程式

在本教學結束時，您將了解 MCP 如何透過提供對更廣泛數位生態系統的存取來增強您的 AI 代理程式，使它們更具能力、情境感知和實用性。

讓我們開始了解 MCP 架構並設定我們的環境！

## MCP 架構概覽

![MCP 架構](../images/mcp_architecture.png)

MCP 遵循客戶端-伺服器架構，具有三個主要元件：

- **主機**：需要存取外部資源的 AI 應用程式（如 Claude Desktop、Cursor 或客製化代理程式）
- **客戶端**：與伺服器維持連線的連接器
- **伺服器**：透過 MCP 協定暴露功能（資料、工具、提示）的輕量級程式
- **資料來源**：MCP 伺服器可以存取的本地（檔案、資料庫）和遠端服務（API）

MCP 內的通訊使用透過 WebSocket 連線的 JSON-RPC 2.0，確保元件之間的即時雙向通訊。

## 體驗 MCP：先試用再建置

雖然本教學專注於建置您自己的 MCP 伺服器並將其與 AI 代理程式整合，但您可能想在深入開發之前快速體驗 MCP 在實務中的運作方式。

官方 MCP 文件為想要在 Claude Desktop 或其他相容的 AI 應用程式中試用現有 MCP 伺服器的使用者提供了優秀的快速入門指南。這讓您無需撰寫任何程式碼就能親身體驗 MCP 啟用的功能。

**👉 親自試試看：** [MCP 使用者快速入門指南](https://modelcontextprotocol.io/quickstart/user)

透過探索快速入門指南，您將對本教學中建置的內容獲得實務洞察。當您準備好了解內部運作並建立自己的實作時，請繼續下面的逐步開發流程。

現在，讓我們開始建置我們自己的 MCP 伺服器和客戶端！

## 建置您的 MCP 伺服器

現在我們了解了 MCP 的基礎知識，讓我們建置我們的第一個 MCP 伺服器！在本節中，我們將使用 CoinGecko API 建立一個加密貨幣價格查詢服務。我們的伺服器將提供允許 AI 檢查加密貨幣當前價格或市場資料的工具。

### 設定我們的環境

在我們深入實作之前，讓我們安裝必要的套件並設定我們的環境。

> **注意：** 對於安裝步驟，請開啟終端機視窗。這些指令應該在一般終端機中執行，而不是在 Jupyter notebook 儲存格中。

#### 步驟 1：安裝 uv 套件管理器

```bash
# 在您的終端機中執行此指令，不是在 Jupyter 中
curl -LsSf https://astral.sh/uv/install.sh | sh
```

#### 步驟 2：設定專案

```bash
# 建立並導航到專案目錄
mkdir mcp-crypto-server
cd mcp-crypto-server
uv init

# 建立並啟動虛擬環境
uv venv
source .venv/bin/activate  # 在 Windows 上：.venv\Scripts\activate

# 安裝相依性
uv add "mcp[cli]" httpx
```

### 執行 MCP 伺服器

在我們設定環境後，我們可以開始建置我們的工具。

請查看 [mcp_server.py](scripts/mcp_server.py) 來了解如何建置工具。

現在我們可以透過在終端機中執行以下指令來啟動伺服器：
```bash
# 從 scripts 資料夾複製伺服器檔案
cp ../scripts/mcp_server.py .

# 啟動 MCP 伺服器 
uv run mcp_server.py
```

### 與 Claude Desktop 整合
如果您還沒有下載 Claude Desktop，請查看[此頁面](https://claude.ai/download)。

要將您的 MCP 伺服器連接到 Claude Desktop：

#### 步驟 1：找到您的 uv 指令的絕對路徑：
```bash
which uv
```
複製輸出（例如，/user/local/bin/uv 或類似路徑）

#### 步驟 2：建立或編輯 Claude Desktop 設定檔：

- 在 macOS 上：~/Library/Application Support/Claude/claude_desktop_config.json
- 在 Windows 上：%APPDATA%\Claude\claude_desktop_config.json
- 在 Linux 上：~/.config/Claude/claude_desktop_config.json

您可以查看[此頁面](https://modelcontextprotocol.io/quickstart/user#2-add-the-filesystem-mcp-server)來了解如何建立設定檔。

#### 步驟 3：新增您的 MCP 伺服器設定：

```json
{
    "mcpServers": {
        "crypto-price-tracker": {
            "command": "/ABSOLUTE/PATH/TO/uv",
            "args": [
                "--directory",
                "/ABSOLUTE/PATH/TO/GenAI_Agents/all_agents_tutorials/mcp-crypto-server",
                "run",
                "mcp_server.py"
            ]
        }
    }
}
```
將 `/ABSOLUTE/PATH/TO/uv` 替換為您從 `which uv` 指令獲得的路徑，並將 `/ABSOLUTE/PATH/TO/GenAI_Agents` 替換為您儲存庫的絕對路徑。


#### 步驟 4：重新啟動 Claude Desktop 以使變更生效。

您應該會在聊天框中看到這個錘子圖示。

![Claude Desktop 與 MCP 連接](../images/Claude_Desktop_with_MCP.png)

#### 步驟 5：嘗試詢問比特幣價格

輸入「What is the current price of Bitcoin ?」，您將得到類似這樣的回應：

![使用 MCP 追蹤比特幣價格](../images/track_bitcoin_price_with_mcp.png)


恭喜！您已成功應用您的 MCP 伺服器和工具。現在，您可以嘗試在 [mcp_server.py](/mcp-crypto-server/mcp_server.py) 中新增您自己的工具。以下是一個範例：

```python
@mcp.tool()
async def get_crypto_market_info(crypto_ids: str, currency: str = "usd") -> str:
    """
    取得一個或多個加密貨幣的市場資訊。
    
    參數：
    - crypto_ids: 加密貨幣 ID 的逗號分隔清單（例如：'bitcoin,ethereum'）
    - currency: 顯示數值的貨幣（預設：'usd'）
    
    回傳：
    - 包含價格、市值、交易量和價格變化的市場資訊
    """
    # 建構 API URL
    url = f"{COINGECKO_BASE_URL}/coins/markets"
    
    # 設定查詢參數
    params = {
        "vs_currency": currency,  # 顯示數值的貨幣
        "ids": crypto_ids,        # 逗號分隔的加密貨幣 ID
        "order": "market_cap_desc", # 按市值排序
        "page": 1,                # 頁碼
        "sparkline": "false"      # 排除 sparkline 資料
    }
    
    try:
        # 進行 API 呼叫
        async with httpx.AsyncClient() as client:
            response = await client.get(url, params=params)
            response.raise_for_status()
            
            # 解析回應
            data = response.json()
            
            # 檢查是否取得任何資料
            if not data:
                return f"找不到加密貨幣的資料：'{crypto_ids}'。請檢查 ID 並重試。"
            
            # 格式化結果
            result = ""
            for crypto in data:
                name = crypto.get('name', 'Unknown')
                symbol = crypto.get('symbol', '???').upper()
                price = crypto.get('current_price', 'Unknown')
                market_cap = crypto.get('market_cap', 'Unknown')
                volume = crypto.get('total_volume', 'Unknown')
                price_change = crypto.get('price_change_percentage_24h', 'Unknown')
                
                result += f"{name} ({symbol}):\n"
                result += f"當前價格: {price} {currency.upper()}\n"
                result += f"市值: {market_cap} {currency.upper()}\n"
                result += f"24小時交易量: {volume} {currency.upper()}\n"
                result += f"24小時價格變化: {price_change}%\n\n"
            
            return result
            
    except Exception as e:
        return f"取得市場資料時發生錯誤: {str(e)}"
```

使用 `uv run mcp_server.py` 重新執行您的 mcp 伺服器，重新啟動 Claude Desktop，然後輸入「What's the market data for Dogecoin and Solana?」。您將得到類似這樣的回應：

![使用 MCP 追蹤加密貨幣市場資料](../images/track_crypto_market_data_with_mcp.png)

## 透過 MCP 執行工具的自訂代理程式

在我們建置自己的 MCP 後，讓我們嘗試自己建置 MCP 主機和客戶端。

### 了解架構

在本節中，我們將建置我們自己的 MCP 主機和客戶端。與之前連接到 Claude Desktop 的方法不同，我們現在將建立一個可以：
1. 作為 MCP 主機
2. 從我們的 MCP 伺服器發現可用的工具
3. 根據使用者查詢了解何時使用哪個工具
4. 使用適當的參數執行工具
5. 處理工具結果以提供有用的回應

此架構遵循現代 AI 系統中常見的模式：
- **發現階段**：我們的自訂主機發現有哪些工具可用
- **規劃階段**：代理程式根據使用者的查詢決定使用哪個工具
- **執行階段**：我們的客戶端連接到伺服器並執行選定的工具
- **解釋階段**：代理程式用自然語言解釋結果

以下是簡單的工作流程圖：

![使用 MCP 追蹤加密貨幣市場資料](../images/customized_mcp_host.png)

執行程式碼前的重要提醒：
⚠️ 別忘了先啟動您的 MCP 伺服器！⚠️
在執行以下教學中的代理程式程式碼之前，請確保您的 MCP 伺服器正在執行。否則，您的代理程式將沒有任何工具可以發現或執行。

讓我們開始設定我們的環境並匯入必要的函式庫：

In [1]:
! pip install mcp anthropic

我們需要兩個主要函式庫：
- **MCP**：處理與我們 MCP 伺服器的客戶端-伺服器通訊，讓我們能夠建置主機和客戶端元件
- **Anthropic**：與 Claude 互動，這將為我們的代理程式提供推理能力

現在，讓我們為我們的代理程式設定必要的匯入和設定：

In [ ]:
# 匯入必要的函式庫
import os
import json
from typing import List, Dict, Any

# 用於連接到伺服器的 MCP 函式庫
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# 用於 Claude 的 Anthropic API
from anthropic import Anthropic

# 設定 Anthropic API 金鑰（使用您提供的金鑰）
os.environ["ANTHROPIC_API_KEY"] = "your_anthropic_api_key_here"

# 初始化 Anthropic 客戶端
client = Anthropic()

# 您的 MCP 伺服器路徑
mcp_server_path = "absolute/path/to/your/running/mcp/server"
print("設定完成！")

Setup complete!


我們使用來自 MCP 的 `stdio_client` 介面，它允許我們連接到作為獨立程序執行並透過標準輸入/輸出通訊的 MCP 伺服器。這是本地開發的簡單且穩健的方法。透過實作 MCP 協定的兩端（主機和客戶端），我們完全控制代理程式如何與 MCP 工具互動。

### 工具發現：建置我們的 MCP 主機

建置我們自訂 MCP 實作的第一步是建立一個可以發現我們 MCP 伺服器有哪些工具可用的主機。我們的主機將作為使用者、AI 和可用工具之間的中介 - 類似於 Claude Desktop 的功能，但在我們的完全控制下。

讓我們實作一個函式來連接到我們的 MCP 伺服器並發現其工具：

In [ ]:
async def discover_tools():
    """
    連接到 MCP 伺服器並發現可用工具。
    回傳可用工具的資訊。
    """
    # 用於更好日誌可見性的 ANSI 顏色代碼
    BLUE = "\033[94m"
    GREEN = "\033[92m"
    RESET = "\033[0m"
    SEP = "=" * 40
    
    # 建立透過 stdio 連接到您的 MCP 伺服器的伺服器參數
    server_params = StdioServerParameters(
        command="python",  # 執行伺服器的指令
        args=[mcp_server_path],  # 您的 MCP 伺服器腳本路徑
    )
    
    print(f"{BLUE}{SEP}\n🔍 發現階段：連接到 MCP 伺服器...{RESET}")
    
    # 透過 stdio 連接到伺服器
    async with stdio_client(server_params) as (read, write):
        # 建立客戶端會話
        async with ClientSession(read, write) as session:
            # 初始化連線
            print(f"{BLUE}📡 初始化 MCP 連線...{RESET}")
            await session.initialize()
            
            # 列出可用工具
            print(f"{BLUE}🔎 發現可用工具...{RESET}")
            tools = await session.list_tools()
            
            # 格式化工具資訊以便於檢視
            tool_info = []
            for tool_type, tool_list in tools:
                if tool_type == "tools":
                    for tool in tool_list:
                        tool_info.append({
                            "name": tool.name,
                            "description": tool.description,
                            "schema": tool.inputSchema
                        })
            
            print(f"{GREEN}✅ 成功發現 {len(tool_info)} 個工具{RESET}")
            print(f"{SEP}")
            return tool_info

print("工具發現函式已定義")

Tool discovery function defined


This function acts as our host's discovery component:

1. **Creates Server Parameters**: Configures how to launch and connect to the MCP server
2. **Establishes Connection**: Uses `stdio_client` to create a communication channel
3. **Initializes Session**: Sets up the MCP session using the communication channel
4. **Discovers Tools**: Calls `list_tools()` to get all available tools
5. **Formats Results**: Converts the tools into a more usable format for our agent

We're using an asynchronous approach (`async/await`) because MCP operations are non-blocking by design. This is important in a host implementation, as it allows our agent to handle multiple operations concurrently and remain responsive even when waiting for tool operations to complete.

Let's test our tool discovery function to make sure it works properly:


In [3]:
# Test the tool discovery function
tools = await discover_tools()
print(f"Discovered {len(tools)} tools:")
for i, tool in enumerate(tools, 1):
    print(f"{i}. {tool['name']}: {tool['description']}")

🔍 DISCOVERY PHASE: Connecting to MCP server...
📡 Initializing MCP connection...
🔎 Discovering available tools...
✅ Successfully discovered 2 tools
Discovered 2 tools:
1. get_crypto_price: 
    Get the current price of a cryptocurrency in a specified currency.
    
    Parameters:
    - crypto_id: The ID of the cryptocurrency (e.g., 'bitcoin', 'ethereum')
    - currency: The currency to display the price in (default: 'usd')
    
    Returns:
    - Current price information as a formatted string
    
2. get_crypto_market_info: 
    Get market information for one or more cryptocurrencies.
    
    Parameters:
    - crypto_ids: Comma-separated list of cryptocurrency IDs (e.g., 'bitcoin,ethereum')
    - currency: The currency to display values in (default: 'usd')
    
    Returns:
    - Market information including price, market cap, volume, and price changes
    


When we run this code, we should see a list of the tools available from our MCP server. In this case, we're expecting to see our cryptocurrency tools.

### Tool Execution: Implementing Our MCP Client

Now that our host can discover available tools, we need to implement the client component that can execute them. Unlike third-party tools that might have this functionality built-in, we're creating our own client to execute MCP tools with complete control and transparency:


In [4]:
async def execute_tool(tool_name: str, arguments: Dict[str, Any]):
    """
    Execute a specific tool provided by the MCP server.
    
    Args:
        tool_name: The name of the tool to execute
        arguments: A dictionary of arguments to pass to the tool
        
    Returns:
        The result from executing the tool
    """
    # ANSI color codes for better log visibility
    BLUE = "\033[94m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RESET = "\033[0m"
    SEP = "-" * 40
    
    server_params = StdioServerParameters(
        command="python",
        args=[mcp_server_path],
    )
    
    print(f"{YELLOW}{SEP}")
    print(f"⚙️ EXECUTION PHASE: Running tool '{tool_name}'")
    print(f"📋 Arguments: {json.dumps(arguments, indent=2)}")
    print(f"{SEP}{RESET}")
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            # Call the specific tool with the provided arguments
            print(f"{BLUE}📡 Sending request to MCP server...{RESET}")
            result = await session.call_tool(tool_name, arguments)
            
            print(f"{GREEN}✅ Tool execution complete{RESET}")
            
            # Format result preview for cleaner output
            result_preview = str(result)
            if len(result_preview) > 150:
                result_preview = result_preview[:147] + "..."
                
            print(f"{BLUE}📊 Result: {result_preview}{RESET}")
            print(f"{SEP}")
            
            return result

print("Tool execution function defined")

Tool execution function defined


This function forms the core of our MCP client:

1. **Connects to Server**: Similar to our discovery function, it establishes a connection to the MCP server
2. **Executes Tool**: Calls the specified tool with the provided arguments
3. **Returns Result**: Gives back whatever the tool returns

Notice that for each tool execution, we create a new connection to the MCP server. While this may seem inefficient, it ensures clean separation between tool calls and avoids potential state issues. This stateless approach simplifies our implementation and makes it more robust. In a production system, you might optimize this by maintaining a persistent connection, but the current approach is excellent for educational purposes as it clearly separates each step in the process.

Now that we have functions to discover and execute tools, we need to integrate these with an AI that can determine when and how to use them. This is where Claude comes in.

### Integrating AI with Our MCP Implementation

With our host and client components in place, we now need to integrate them with an AI system that can make intelligent decisions about tool usage. This is the "brains" of our custom MCP host, and it needs to:
1. Understand when a tool is needed based on user input
2. Choose the appropriate tool for the task
3. Format the arguments correctly
4. Process and explain the results

Let's implement a function that orchestrates this entire process:

In [5]:
async def query_claude(prompt: str, tool_info: List[Dict], previous_messages=None):
    """
    Send a query to Claude and process the response.
    
    Args:
        prompt: User's query
        tool_info: Information about available tools
        previous_messages: Previous messages for maintaining context
        
    Returns:
        Claude's response, potentially after executing tools
    """
    # ANSI color codes for better log visibility
    BLUE = "\033[94m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    PURPLE = "\033[95m"
    RESET = "\033[0m"
    SEP = "=" * 40
    
    if previous_messages is None:
        previous_messages = []
    
    print(f"{PURPLE}{SEP}")
    print("🧠 REASONING PHASE: Processing query with Claude")
    print(f"🔤 Query: \"{prompt}\"")
    print(f"{SEP}{RESET}")
    
    # Format tool information for Claude
    tool_descriptions = "\n\n".join([
        f"Tool: {tool['name']}\nDescription: {tool['description']}\nSchema: {json.dumps(tool['schema'], indent=2)}"
        for tool in tool_info
    ])
    
    # Build the system prompt
    system_prompt = f"""You are an AI assistant with access to specialized tools through MCP (Model Context Protocol).
    
Available tools:
{tool_descriptions}

When you need to use a tool, respond with a JSON object in the following format:
{{
    "tool": "tool_name",
    "arguments": {{
        "arg1": "value1",
        "arg2": "value2"
    }}
}}

Do not include any other text when using a tool, just the JSON object.
For regular responses, simply respond normally.
"""
    
    # Filter out system messages from previous messages
    filtered_messages = [msg for msg in previous_messages if msg["role"] != "system"]
    
    # Build the messages for the conversation (WITHOUT system message)
    messages = filtered_messages.copy()
    
    # Add the current user query
    messages.append({"role": "user", "content": prompt})
    
    print(f"{BLUE}📡 Sending request to Claude API...{RESET}")
    
    # Send the request to Claude with system as a top-level parameter
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=4000,
        system=system_prompt,  # System prompt as a separate parameter
        messages=messages      # Only user and assistant messages
    )
    
    # Get Claude's response
    claude_response = response.content[0].text
    print(f"{GREEN}✅ Received response from Claude{RESET}")
    
    # Try to extract and parse JSON from the response
    try:
        # Look for JSON pattern in the response
        import re
        json_match = re.search(r'(\{[\s\S]*\})', claude_response)
        
        if json_match:
            json_str = json_match.group(1)
            print(f"{YELLOW}🔍 Tool usage detected in response{RESET}")
            print(f"{BLUE}📦 Extracted JSON: {json_str}{RESET}")
            
            tool_request = json.loads(json_str)
            
            if "tool" in tool_request and "arguments" in tool_request:
                tool_name = tool_request["tool"]
                arguments = tool_request["arguments"]
                
                print(f"{YELLOW}🔧 Claude wants to use tool: {tool_name}{RESET}")
                
                # Execute the tool using our MCP client
                tool_result = await execute_tool(tool_name, arguments)
                
                # Convert tool result to string if needed
                if not isinstance(tool_result, str):
                    tool_result = str(tool_result)
                
                # Update messages with the tool request and result
                messages.append({"role": "assistant", "content": claude_response})
                messages.append({"role": "user", "content": f"Tool result: {tool_result}"})
                
                print(f"{PURPLE}🔄 Getting Claude's interpretation of the tool result...{RESET}")
                
                # Get Claude's interpretation of the tool result
                final_response = client.messages.create(
                    model="claude-3-5-sonnet-20240620",
                    max_tokens=4000,
                    system=system_prompt,
                    messages=messages
                )
                
                print(f"{GREEN}✅ Final response ready{RESET}")
                print(f"{SEP}")
                
                return final_response.content[0].text, messages
        
    except (json.JSONDecodeError, KeyError, AttributeError) as e:
        print(f"{YELLOW}⚠️ No tool usage detected in response: {str(e)}{RESET}")
    
    print(f"{GREEN}✅ Response ready{RESET}")
    print(f"{SEP}")
    
    return claude_response, messages

print("Claude query function defined")

Claude query function defined


This function completes our custom MCP host implementation with a sophisticated reasoning and execution flow:

1. **Tool Description**: We format the tool information in a way Claude can understand
2. **System Prompt**: We provide instructions on when and how to use tools
3. **Response Analysis**: We look for JSON tool requests in Claude's responses
4. **Tool Execution**: If a tool request is detected, we use our client to execute the appropriate tool
5. **Result Processing**: We send the tool results back to Claude for interpretation
6. **Conversation Management**: We maintain context by tracking messages

This creates a powerful synergy: Claude provides the reasoning and communication skills, while our MCP tools provide specialized capabilities and real-time data access.

Let's test our agent with a simple query about Bitcoin prices:

In [ ]:
# Run a single query using the tools from your MCP server
query = "What is the current price of Bitcoin?"
print(f"Sending query: {query}")

response, messages = await query_claude(query, tools)
print(f"\nAssistant's response:\n{response}")


Sending query: What is the current price of Bitcoin?
🧠 REASONING PHASE: Processing query with Claude
🔤 Query: "What is the current price of Bitcoin?"
📡 Sending request to Claude API...
✅ Received response from Claude
🔍 Tool usage detected in response
📦 Extracted JSON: {
    "tool": "get_crypto_price",
    "arguments": {
        "crypto_id": "bitcoin"
    }
}
🔧 Claude wants to use tool: get_crypto_price
----------------------------------------
⚙️ EXECUTION PHASE: Running tool 'get_crypto_price'
📋 Arguments: {
  "crypto_id": "bitcoin"
}
----------------------------------------
📡 Sending request to MCP server...
✅ Tool execution complete
📊 Result: meta=None content=[TextContent(type='text', text='The current price of bitcoin is 83667 USD', annotations=None)] isError=False
----------------------------------------
🔄 Getting Claude's interpretation of the tool result...
✅ Final response ready

Assistant's response:
Based on the tool result, I can provide you with the current price of Bitcoin

When we run this query, our complete MCP implementation follows this flow:
1. Claude (via our host) recognizes this as a request about Bitcoin prices
2. Our AI decides to use the `get_crypto_price` tool
3. It formats the arguments correctly (using "bitcoin" as the crypto_id)
4. Our client connects to the server and executes the tool, returning the current Bitcoin price
5. Claude explains the result in natural language with additional context

This demonstrates the full capability of our agent: understanding the user's intent, selecting the appropriate tool, executing it correctly, and providing a helpful, context-rich response.

### Direct Tool Execution via Our Client

While our integrated MCP host typically decides which tools to use based on the user's query, sometimes we might want to directly use our client to execute a specific tool. This is useful for testing our client implementation or demonstrating specific tool functionality. Let's create a simple example:

In [ ]:
try:
    # Get the first tool name from your discovered tools
    if tools:
        first_tool = tools[0]
        tool_name = first_tool["name"]
        
        # Use the correct parameter name for get_crypto_price
        arguments = {"crypto_id": "bitcoin"}
        
        print(f"Executing tool '{tool_name}' with arguments: {arguments}")
        result = await execute_tool(tool_name, arguments)
        print(f"Tool result: {result}")
    else:
        print("No tools discovered to test")
except Exception as e:
    print(f"Error executing tool: {str(e)}")

Executing tool 'get_crypto_price' with arguments: {'crypto_id': 'bitcoin'}
----------------------------------------
⚙️ EXECUTION PHASE: Running tool 'get_crypto_price'
📋 Arguments: {
  "crypto_id": "bitcoin"
}
----------------------------------------
📡 Sending request to MCP server...
✅ Tool execution complete
📊 Result: meta=None content=[TextContent(type='text', text='The current price of bitcoin is 83670 USD', annotations=None)] isError=False
----------------------------------------
Tool result: meta=None content=[TextContent(type='text', text='The current price of bitcoin is 83670 USD', annotations=None)] isError=False


This direct execution approach is useful for:
- Testing our client implementation in isolation
- Debugging tool functionality
- Building specialized workflows where tool execution is predetermined
- Verifying that our MCP client works correctly before integrating it with the AI

Now, let's create an interactive chat interface that uses our complete MCP host implementation:

### Building an Interactive MCP Host Interface

For a complete MCP host implementation, we need a user interface that maintains context across multiple turns of conversation. This allows our host to remember previous interactions and build on them in subsequent exchanges, just like professional MCP hosts such as Claude Desktop. Let's implement a simple chat session function:

In [ ]:
async def chat_session():
    """
    Run an interactive chat session with the AI agent.
    """
    # ANSI color codes for better log visibility
    BLUE = "\033[94m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    CYAN = "\033[96m"
    BOLD = "\033[1m"
    RESET = "\033[0m"
    SEP = "=" * 50
    
    print(f"{CYAN}{BOLD}{SEP}")
    print("🤖 INITIALIZING MCP AGENT")
    print(f"{SEP}{RESET}")
    
    # Make sure 'tools' is defined from a previous cell, or discover them again
    try:
        # Check if tools is defined and not empty
        if 'tools' not in globals() or not tools:
            print(f"{BLUE}🔍 No tools found, discovering available tools...{RESET}")
            tools_local = await discover_tools()
        else:
            tools_local = tools
            
        print(f"{GREEN}✅ Agent ready with {len(tools_local)} tools:{RESET}")
        
        # Print the available tools for reference
        for i, tool in enumerate(tools_local, 1):
            print(f"{YELLOW}  {i}. {tool['name']}{RESET}")
            print(f"     {tool['description'].strip()}")
        
        # Start the chat session
        print(f"\n{CYAN}{BOLD}{SEP}")
        print(f"💬 INTERACTIVE CHAT SESSION")
        print(f"{SEP}")
        print(f"Type 'exit' or 'quit' to end the session{RESET}")
        
        messages = []
        
        while True:
            # Get user input
            user_input = input(f"\n{BOLD}You:{RESET} ")
            
            # Check if user wants to exit
            if user_input.lower() in ['exit', 'quit']:
                print(f"\n{GREEN}Ending chat session. Goodbye!{RESET}")
                break
            
            # Process the query with Claude
            print(f"\n{BLUE}Processing...{RESET}")
            response, messages = await query_claude(user_input, tools_local, messages)
            
            # Display Claude's response
            print(f"\n{BOLD}Assistant:{RESET} {response}")
            
    except Exception as e:
        print(f"\n{YELLOW}⚠️ An error occurred: {str(e)}{RESET}")

print("Chat session function defined. Run 'await chat_session()' in the next cell to start chatting.")

Chat session function defined. Run 'await chat_session()' in the next cell to start chatting.


Our MCP host interface:

1. **Initializes Tools**: Our host discovers available tools when starting
2. **Creates a Session Loop**: Continuously prompts for user input
3. **Maintains Context**: Passes previous messages to each query, maintaining stateful conversations
4. **Handles Graceful Exit**: Allows the user to end the session gracefully

This creates a natural, conversational experience where the agent can remember previous interactions. For example, if a user asks about Bitcoin and then follows up with "How about Ethereum?", the agent understands the context.

Now, let's run our chat session to see the complete agent in action:

You may try what we ask in Clude Desktop: What's the market data for Dogecoin and Solana?

In [ ]:
# Run the chat session
await chat_session()

🤖 INITIALIZING MCP AGENT
✅ Agent ready with 2 tools:
  1. get_crypto_price
     Get the current price of a cryptocurrency in a specified currency.
    
    Parameters:
    - crypto_id: The ID of the cryptocurrency (e.g., 'bitcoin', 'ethereum')
    - currency: The currency to display the price in (default: 'usd')
    
    Returns:
    - Current price information as a formatted string
  2. get_crypto_market_info
     Get market information for one or more cryptocurrencies.
    
    Parameters:
    - crypto_ids: Comma-separated list of cryptocurrency IDs (e.g., 'bitcoin,ethereum')
    - currency: The currency to display values in (default: 'usd')
    
    Returns:
    - Market information including price, market cap, volume, and price changes

💬 INTERACTIVE CHAT SESSION
Type 'exit' or 'quit' to end the session

Processing...
🧠 REASONING PHASE: Processing query with Claude
🔤 Query: "What's the market data for Dogecoin and Solana?"
📡 Sending request to Claude API...
✅ Received response from

With this final piece, we've created a complete custom MCP host implementation that can:
1. Connect to an MCP server as a client
2. Discover available tools
3. Intelligently select and use those tools to answer user queries
4. Maintain context across a conversation

This demonstrates the power of implementing our own MCP host and client - we get complete control over how AI interacts with tools while maintaining all the benefits of the MCP protocol's standardization.

## 結論：

模型內容協定代表了一種變革性的方法來整合 AI 模型與外部資源，解決了 AI 應用程式開發中的關鍵挑戰：

### 協定優勢
- **標準化整合**：消除複雜的客製化 API 連線
- **動態工具發現**：讓 AI 能夠無縫找到並使用工具
- **靈活通訊**：支援即時雙向互動

### 技術亮點
我們的實作展示了：
- 建置具有專門工具的 MCP 伺服器
- 建立可以動態發現和執行工具的主機
- 將 AI 模型與外部資源整合

### 重要參考資料

- [Python MCP SDK](https://github.com/modelcontextprotocol/python-sdk)
- [MCP 快速入門指南](https://modelcontextprotocol.io/quickstart/user)
- [CoinGecko API 文件](https://www.coingecko.com/en/api)